# Dataset en Verwerking

Voor deze opdracht maken wij gebruik van drie verschillende datasets van verschillende bronnen. Deze worden in dit onderdeel beschreven en verwerkt. Ten slotte worden de datasets samengevoegd tot één dataset.

Voor deze opdracht maken wij gebruik van de volgende datasets:
- Persvrijheid dataset van Reporters Without Borders (RSF) (https://rsf.org/en/index) waarvan alleen data wordt gebruikt uit 2019 en 2023.
- Democratie dataset van Our World In Data (https://ourworldindata.org/grapher/democracy-index-eiu) waarvan alleen data wordt gebruikt uit 2019 en 2023.
- Conflict Index dataset van ACLED (https://acleddata.com/conflict-index/) hier is alleen data van 2019 en 2023 beschikbaar.

In [1]:
import pandas as pd

from plotly.offline import init_notebook_mode
from IPython.display import display

init_notebook_mode(connected=True)

## Persvrijheid dataset

Voor de persvrijheid data maken we gebruik van de datasets van Reporters Without Borders (RSF). Deze datasets bevatten informatie over de persvrijheid in verschillende landen, waaronder de persvrijheidsscore en de ranglijst van elk land. De datasets die wij gebruiken zijn van 2019 en 2023.

### Persvrijheid Dataset Schoonmaken

De persvrijheid dataset van 2019 verschilt met die van 2023, dus worden deze eerst apart ingeladen en schoongemaakt. Daarna worden deze samengevoegd.

#### 2019 Dataset

Beschrijving over variabelen...

In [2]:
persvrijheid_oud = pd.read_csv('./data/persvrijheid/2019.csv', sep=';')

# Hernoemen van kolommen om deze makkelijker te kunnen gebruiken.
persvrijheid_oud = persvrijheid_oud.rename(
    columns={
        "Year (N)": "jaar",
        "ISO": "iso",
        "EN_country": "land",
        "Rank N": "rang",
        "Score N": "score",
    })

# Verwijderen van onnodige kolommen.
persvrijheid_oud = persvrijheid_oud.drop(columns=[
    "Score N without the exactions",
    "Score N with the exactions",
    "Rank N-1",
    "Rank evolution",
    "Score exactions",
    "Score N-1",
    "Zone", # De data uit deze kolom is niet door alle jaren heen hetzelfde, dus deze wordt verwijderd.
    "jaar",
    "land", # We maken gebruik van de 2023 landnamen.
])

# Verwijder kolommen met buitenlandse namen van landen.
persvrijheid_oud = persvrijheid_oud.drop(columns=[
    col
    for col in persvrijheid_oud.columns
    if "country" in col.lower()
])

display(persvrijheid_oud.head())

,iso,rang,score
0,NOR,1,"92,18"
1,FIN,2,"92,1"
2,SWE,3,"91,69"
3,NLD,4,"91,37"
4,DNK,5,"90,13"


#### 2023 Dataset

Beschrijving over variabelen...

In [3]:
persvrijheid_nieuw = pd.read_csv('./data/persvrijheid/2023.csv', sep=';')

# Hernoemen van kolommen om deze makkelijker te kunnen gebruiken.
persvrijheid_nieuw = persvrijheid_nieuw.rename(
    columns={
        "Year (N)": "jaar",
        "ISO": "iso",
        "Country_EN": "land",
        "Rank": "rang",
        "Score": "score",
    })

# Verwijderen van onnodige kolommen.
persvrijheid_nieuw = persvrijheid_nieuw.drop(columns=[
    "Political Context", "Rank_Pol",
    "Economic Context", "Rank_Eco",
    "Legal Context", "Rank_Leg",
    "Social Context", "Rank_Soc",
    "Safety", "Rank_Saf",
    "Rank N-1",
    "Rank evolution",
    "Score N-1",
    "Score evolution",
    "Zone", # De data uit deze kolom is niet door alle jaren heen hetzelfde, dus deze wordt verwijderd.
    "jaar",
])

# Verwijder kolommen met buitenlandse namen van landen.
persvrijheid_nieuw = persvrijheid_nieuw.drop(columns=[
    col
    for col in persvrijheid_nieuw.columns
    if "country" in col.lower()
])

display(persvrijheid_nieuw.head())

,iso,score,rang,land
0,NOR,"95,18",1,Norway
1,IRL,"89,91",2,Ireland
2,DNK,"89,48",3,Denmark
3,SWE,"88,15",4,Sweden
4,FIN,"87,94",5,Finland


### Persvrijheid Dataset Samenvoegen

De datasets van 2019 en 2023 worden samengevoegd tot één dataset.

Beschrijving over variabelen... 

In [4]:
# Datasets samenvoegen op basis van de ISO code van het land.
persvrijheid_df = pd.merge(
    persvrijheid_oud,
    persvrijheid_nieuw,
    on="iso",
    how="inner",
    suffixes=('_2019', '_2023')
)

# Herordenen van de kolommen. (Geen functionele waarde)
persvrijheid_df = persvrijheid_df[[
    "iso",
    "land",
    "score_2019",
    "rang_2019",
    "score_2023",
    "rang_2023",
]]

display(persvrijheid_df.head())

,iso,land,score_2019,rang_2019,score_2023,rang_2023
0,NOR,Norway,"92,18",1,"95,18",1
1,FIN,Finland,"92,1",2,"87,94",5
2,SWE,Sweden,"91,69",3,"88,15",4
3,NLD,Netherlands,"91,37",4,87,6
4,DNK,Denmark,"90,13",5,"89,48",3


## Democratie dataset

Voor de democratie data maken we gebruik van de datasets van Our World in Data. Deze datasets bevatten informatie over de democratie in verschillende landen, waaronder de democratie score en de ranglijst van elk land. De dataset bevat data van 2006 tot 2023, maar wij gebruiken alleen de data van 2019 en 2023.


### Democratie Dataset Schoonmaken

In [5]:
democratie_df = pd.read_csv('./data/democracy.csv')

# Hernoemen van de kolommen om deze makkelijker te kunnen gebruiken.
democratie_df = democratie_df.rename(
    columns={
        "Entity": "land",
        "Code": "iso",
        "Year": "jaar",
        "Democracy score": "score",
    })

# Verwijder landen waarvan geen ISO code bekend is.
# Dit zijn geen landen, maar regio's.
democratie_df = democratie_df.dropna(subset=["iso"])

# Verwijder landen waarvan de ISO code geen ISO3 code is.
# Dit zijn in deze dataset alleen de gemiddelde scores van de wereld.
democratie_df = democratie_df[democratie_df["iso"].apply(lambda x: len(x) == 3)]

# Dataset herinrichten in kolommen voor scores van 2019 en 2023.
democratie_df = pd.merge(
    democratie_df[democratie_df["jaar"] == 2019],
    democratie_df[democratie_df["jaar"] == 2023],
    on="iso",
    how="inner",
    suffixes=('_2019', '_2023')
)

# Verwijderen van onnodige kolommen.
democratie_df = democratie_df.drop(columns=[
    "jaar_2019",
    "jaar_2023",
    "land_2019",
])

# Hernoemen van de 'land_2023' kolom naar 'land'.
democratie_df = democratie_df.rename(columns={"land_2023": "land"})

# Herordenen van de kolommen. (Geen functionele waarde)
democratie_df = democratie_df[[
    "iso",
    "land",
    "score_2019",
    "score_2023",
]]

display(democratie_df.head())

,iso,land,score_2019,score_2023
0,ALB,Albania,5.89,6.28
1,DZA,Algeria,4.01,3.66
2,AGO,Angola,3.72,4.18
3,ARG,Argentina,7.02,6.62
4,ARM,Armenia,5.54,5.42


### Democratie Rangschikken

Ieder land krijgt een ranking gebaseerd op de democratie score. Landen met de hoogste score krijgen de rang `1`. Landen met dezelfde score krijgen dezelfde ranking. De rangschikking voor ieder land wordt per jaar berekend.

In [6]:
def rangschik_democratie(df):
    new_df = df.copy()

    # Maak een geordende lijst van unieke scores.
    # Op basis van de index van een score in deze lijst wordt de rang bepaald.
    unieke_dem_scores_2019 = sorted(list(df["score_2019"].unique()), reverse=True)
    unieke_dem_scores_2023 = sorted(list(df["score_2023"].unique()), reverse=True)
    
    # Bereken de rangschikking van de landen.
    new_df["rang_2019"] = new_df["score_2019"].apply(lambda x: unieke_dem_scores_2019.index(x) + 1)
    new_df["rang_2023"] = new_df["score_2023"].apply(lambda x: unieke_dem_scores_2023.index(x) + 1)
    
    return new_df

democratie_df = rangschik_democratie(democratie_df)

display(democratie_df.head())

,iso,land,score_2019,score_2023,rang_2019,rang_2023
0,ALB,Albania,5.89,6.28,71,58
1,DZA,Algeria,4.01,3.66,102,100
2,AGO,Angola,3.72,4.18,107,97
3,ARG,Argentina,7.02,6.62,44,46
4,ARM,Armenia,5.54,5.42,78,75


## Conflict Index dataset

Voor de conflict data maken we gebruik van de datasets van The Armed Conflict Location & Event Data Project (ACLED). Deze datasets bevatten informatie over conflicten in verschillende landen, waaronder de conflict rangschikking van elk land. De dataset die wij gebruiken heeft data vanaf het jaar 2023, en vergelijkt de data van een land met de data van het land uit 2019. Uit de vergelijkingen met 2019 worden de conflict index scores van 2019.

In deze dataset zijn er geen conflict scores van 2019 beschikbaar, dus is het niet mogelijk om deze scores te vergelijken met 2023. De conflict score van 2023 wordt daarom ook niet gebruikt in de analyse. Ditzelfde wordt gedaan voor de conflict categorie, omdat hier geen duidelijke vergelijking mee gemaakt kan worden tussen 2019 en 2023.

Beschriving over variabelen in de dataset...

### Conflict Index Dataset Schoonmaken

In [7]:
conflict_df = pd.read_csv('./data/conflict.csv')

# Hernoemen van kolommen om deze makkelijker te kunnen gebruiken.
conflict_df = conflict_df.rename(columns={
    "country": "land",
    "Index Ranking 2023": "rang_2023",
    "Change in ranking since 2019": "rang_verandering",
    "Index Category 2023": "categorie_2023",
})


# Verwijderen van onnodige kolommen.
conflict_df = conflict_df[[
    "land",
    "rang_2023",
    "rang_verandering",
    "categorie_2023",
]]

# Berekenen van de rangschikking van 2019.
conflict_df["rang_2019"] = conflict_df["rang_2023"] + conflict_df["rang_verandering"]

# De kolom 'rang_verandering' is niet meer nodig.
conflict_df = conflict_df.drop(columns=["rang_verandering"])

### Conflict Index Rangschikken

In de Conflict Index dataset is al een rangschikking aanwezig, maar deze rangschikking geeft een land met een hoge mate van conflicten en/of geweld een hoge rang. Het land met de hoogste maat van conflict en/of geweld krijgt namelijk de rang van `1`. Omdat in deze opdracht bij de andere datasets een hoge rang een positieve score betekent, wordt de rangschikking van de Conflict Index dataset omgedraaid. Een land met de laagste conflict index rang krijgen de rang `1`. De rangschikking voor ieder land wordt per jaar berekend.

In [8]:
def rangschik_conflict(df):
    new_df = df.copy()

    # Omdraaien van de rangschikking.
    new_df["rang_2019"] = new_df["rang_2019"].max() - new_df["rang_2019"] + 1
    new_df["rang_2023"] = new_df["rang_2023"].max() - new_df["rang_2023"] + 1

    return new_df

conflict_df = rangschik_conflict(conflict_df)

display(conflict_df.head())

,land,rang_2023,categorie_2023,rang_2019
0,Myanmar,163,Extreme,116
1,Syria,162,Extreme,133
2,Palestine,161,Extreme,111
3,Mexico,160,Extreme,131
4,Nigeria,159,Extreme,123


## Samenvoegen van datasets

De datasets van persvrijheid, democratie en conflict index worden samengevoegd tot één dataset. De datasets worden samengevoegd op basis van de ISO code van het land.

De rangen en/of scores van de datasets krijgen hierbij een prefix om aan te geven bij welke dataset de rang en/of score hoort, bijvoorbeeld 'persvrijheid_rang_2023' voor de rang van de persvrijheid dataset van een land in 2023.

In [9]:
# Toevoegen van prefixes aan de kolomnamen, als deze er nog niet zijn.
def add_prefix(df, prefix):
    new_df = df.copy()
    for col in new_df.columns:
        if not col.startswith(prefix):
            new_df = new_df.rename(columns={col: prefix+col})
    return new_df

persvrijheid_df = add_prefix(persvrijheid_df, "persvrijheid_")
democratie_df = add_prefix(democratie_df, "democratie_")
conflict_df = add_prefix(conflict_df, "conflict_")

# Samenvoegen van de datasets.
eind_df = pd.merge(
    persvrijheid_df,
    democratie_df,
    left_on="persvrijheid_iso",
    right_on="democratie_iso",
    how="inner"
)

# Verwijderen van de dubbele kolommen.
eind_df = eind_df.drop(columns=[
    "democratie_iso",
    "democratie_land",
])

# Hernoemen van kolommen.
eind_df = eind_df.rename(columns={
    "persvrijheid_iso": "iso",
    "persvrijheid_land": "land",
})

# Sommige landen namen komen niet in alle datasets voor, of zijn niet hetzelfde.
# De landen die niet een conflict index hebben worden hier weergegeven.
# display(set(eind_df["land"]) - set(conflict_df["conflict_land"]))

# Dataset samenvoegen met de conflict dataset op basis van land.
eind_df = pd.merge(
    eind_df,
    conflict_df,
    left_on="land",
    right_on="conflict_land",
    how="inner"
)

# Verwijderen van de dubbele kolommen.
eind_df = eind_df.drop(columns=[
    "conflict_land",
])

display(eind_df)

,iso,land,persvrijheid_score_2019,persvrijheid_rang_2019,persvrijheid_score_2023,persvrijheid_rang_2023,democratie_score_2019,democratie_score_2023,democratie_rang_2019,democratie_rang_2023,conflict_rang_2023,conflict_categorie_2023,conflict_rang_2019
0,NOR,Norway,"92,18",1,"95,18",1,9.87,9.81,1,1,18,Low/Inactive,1
1,FIN,Finland,"92,1",2,"87,94",5,9.25,9.30,5,5,18,Low/Inactive,1
2,SWE,Sweden,"91,69",3,"88,15",4,9.39,9.39,3,4,24,Low/Inactive,1
3,NLD,Netherlands,"91,37",4,87,6,9.01,9.00,10,9,39,Low/Inactive,1
4,DNK,Denmark,"90,13",5,"89,48",3,9.22,9.28,7,6,1,Low/Inactive,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,BHR,Bahrain,"38,69",167,"30,59",171,2.55,2.52,130,127,32,Low/Inactive,23
135,CUB,Cuba,"36,19",169,29,172,2.84,2.65,125,123,77,Low/Inactive,23
136,DJI,Djibouti,"28,64",173,"35,87",162,2.77,2.70,126,122,37,Low/Inactive,33
137,VNM,Vietnam,"25,07",176,"24,58",178,3.08,2.62,120,124,53,Low/Inactive,27


### Veranderingen

Deze kolommen worden gebruikt bij visualisaties om de verandering van de rangen in de verschillende datasets te visualiseren.

In [12]:
# Voeg 3 extra kolommen toe met 
eind_df['persvrijheid_verandering'] = eind_df['persvrijheid_rang_2019'] - eind_df['persvrijheid_rang_2023']
eind_df['conflict_verandering'] = eind_df['conflict_rang_2019'] - eind_df['conflict_rang_2023']
eind_df['democratie_verandering'] = eind_df['democratie_rang_2019'] - eind_df['democratie_rang_2023']

## Dataset opslaan

De samengevoegde dataset wordt opgeslagen in Pickle bestand, zodat deze in een ander notebook gebruikt kan worden.

In [11]:
eind_df.to_pickle('./data/dataset.pkl')